# Stencil Calculations with Numba

In [ ]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## Heat equation
\begin{align}
\frac{\partial u}{\partial t} &= \alpha\frac{\partial^2u}{\partial x^2},\quad\quad &\text{$x\in (0,L), t \in (0, T]$} \\
u(x,0)&=I_0(x),\quad\quad &\text{$x \in [0, L]$} \\
u(0,t) &= 0,\quad\quad & t>0 \\
u(L,t) &= 0,\quad\quad & t>0
\end{align}

## Discretization of the Heat equation

\begin{align}
\text{space discretization:}\quad\quad\quad\quad\quad x_i &=i \Delta x, \quad i=0,\ldots,Nx\\
\text{time discretization:}\quad\quad\quad\quad\quad t_n &= n\Delta t, \quad n = 0,\ldots, N_t \\
\frac{u_i^{n+1}-u_i^n}{\Delta t} &= \alpha \frac{u_{i+1}^n-2u_i^n+u_{i-1}^n}{\Delta x^2}\\
u_i^{n+1} &= u_i^n + F \left( u_{i+1}^n -2 u_i^n + u_{i-1}^n\right) \\
\text{mesh Fourier number:}\quad\quad\quad\quad\quad F &= \alpha \frac{\Delta t}{\Delta x ^2}
\end{align}

In [ ]:
nx = 500
nt = 10000
T = 100.0
L = 100.0
dx = L / nx
dt = T / nt
alpha = 1.0
F = alpha * dt / dx ** 2 
print(F)
x = np.linspace(0, L, nx + 1)
temp = np.zeros(nx + 1)
temp[nx//2:nx//2+nx//10] = np.sin(2.0 * np.pi * (x[nx//2:nx//2+nx//10] - x[nx//2])/(L/5))

In [ ]:
@numba.stencil
def central_diff_stencil(x, F):
    return x[0]  + F * (x[1] - 2.0 * x[0] + x[-1])

In [ ]:
@numba.njit(parallel=True)
def simulation(x, F, nt):
    for i in range(nt):
        x = central_diff_stencil(x, F)
    
    return x

In [ ]:
temp_final = simulation(temp, F, nt)
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
ax.plot(x, temp, label='t=0.0')
ax.plot(x, temp_final, label=f't={T}')
ax.set_xlabel('x')
ax.set_ylabel('u')
ax.legend();